In [1]:
import pandas as pd
import numpy as np
import os, sys, inspect
import folium
import subprocess
from folium.plugins import HeatMap
from sklearn import preprocessing

# add parent dir to system dir
currdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
rootdir = os.path.dirname(currdir)
sys.path.insert(0, rootdir)

from src.utils import get_temporal_data, get_spatial_data

In [2]:
ffname_realestate = os.path.join(rootdir, "data", "int", "realestate_demographics.csv")
realestate_df = pd.read_csv(ffname_realestate)
#realestate_df.head()

In [3]:
# ziptrend = get_temporal_data(78701,realestate_df)

# ziptrend.head()

In [4]:
fname = "Home Value Index"
yeartrend = get_spatial_data(2017,realestate_df)
yeartrend12 = yeartrend.loc[yeartrend["Month"]==12,:]
#yeartrend12.head()


In [5]:
column_to_heatmap = "Home Value Index"
yeartrend_df = yeartrend12[["Latitude","Longitude",column_to_heatmap]]


In [6]:
yeartrend_df.reset_index(inplace=True)
yeartrend_df.head()

,index,Latitude,Longitude,Home Value Index
0,260,30.272921,-97.744386,407200.0
1,525,30.260353,-97.714515,399900.0
2,790,30.291533,-97.768836,790000.0
3,1055,30.245673,-97.768836,486100.0
4,1320,30.296171,-97.738954,241600.0


In [7]:
yeartrend_df.drop(["index"], inplace=True, axis=1)
yeartrend_df.head()

C:\Users\mywor\AppData\Local\conda\conda\envs\PythonData\lib\site-packages\ipykernel\__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


,Latitude,Longitude,Home Value Index
0,30.272921,-97.744386,407200.0
1,30.260353,-97.714515,399900.0
2,30.291533,-97.768836,790000.0
3,30.245673,-97.768836,486100.0
4,30.296171,-97.738954,241600.0


In [17]:
data = yeartrend_df.values

minmax_scalar = preprocessing.MinMaxScaler()
data[:,[2]] = minmax_scalar.fit_transform(data[:,[2]])

In [12]:
max_val = yeartrend_df["Home Value Index"].max()
min_val = yeartrend_df["Home Value Index"].min()
threshold_scale = np.linspace(min_val, max_val, 6)

In [13]:
yeartrend_df["Home Value Index"].max()

873300.0

In [14]:
threshold_scale.tolist()

[183800.0, 321700.0, 459600.0, 597500.0, 735400.0, 873300.0]

In [15]:
# heatmap params

location = [30.3272, -97.7431]
zoom_start = 11
name = column_to_heatmap
radius = 30

# create map
m = folium.Map(location, zoom_start=zoom_start)
ffname = os.path.join(rootdir, "data", "ext", "austin-zipcode-boundaries.geojson")
m.choropleth(geo_data=ffname, data=yeartrend_df,
            columns=['Latitude','Longitude'],
            key_on="Home Value Index",
            threshold_scale=threshold_scale.tolist(),
            fill_color='BuPu')
m

In [ ]:
#help(m.choropleth)

In [18]:
HeatMap(data.tolist(),radius=radius,min_opacity = .8, blur = 45, gradient = {0.4: 'blue', 0.65: 'lime', 1: 'red'}).add_to(m)
m.save(os.path.join(rootdir, 'Reports', 'Figures', (name + '.html')))

In [ ]:
#help(HeatMap)

In [ ]:
#help(folium.Map)